In [3]:
import pandas as pd
import pymc3 as pm
import numpy as np
from scipy.stats import gamma

from empiricaldist import Pmf
import matplotlib.pyplot as plt

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [4]:
# Load data from results.csv
results = pd.read_csv('../data/results.csv')
teams = pd.read_csv('../data/teams.csv')
fixtures = pd.read_csv('../data/fixtures.csv')
# Define the outcome variable
results['Outcome'] = results.apply(
    lambda row: 'HomeWin' if row['HomeScore'] > row['AwayScore'] 
    else ('Draw' if row['HomeScore'] == row['AwayScore'] else 'AwayWin'), axis=1
)

In [5]:
def results_points(a, b):
    """ Win, lose, draw 3, 0, 1 """
    if a > b:
        return [3, 0]
    if b > a:
        return [0, 3]
    if b == a:
        return [1, 1]
    else:
        return ValueError

In [6]:
def get_points_by_week(df):
    new_df = df.copy()
    home = new_df[['Gameweek', 'HomeTeamID', 'HomePoints', 'HomeScore', 'AwayScore']].copy()
    away = new_df[['Gameweek', 'AwayTeamID', 'AwayPoints', 'AwayScore', 'HomeScore']].copy()
    cols = ['Gameweek', 'ID', 'Points', 'For', 'Against']
    home.columns = cols
    away.columns = cols
    points_by_week = pd.concat([home, away])
    return points_by_week

In [7]:
# for fixture - get fixture
fixture_predict = fixtures[fixtures.index==0]
def get_goals_data(selected_fixture, results_data):
    """Given a fixture row from the data
    filter the data to return the head to head records for
    the teams and then give vectors for their goal scoring
    # TODO: (1) remove head to from all fixtures - remove double count
    (2) Treats home and away as the same - needs to factor this in - create separate model?
    (3) Doesn't use the data for shots taken - can be added into score as XG
    (4) Doesn't account for goals conceded - !! need to look up
    """
    # Assign team A home, and team B away 
    teams_ab = [selected_fixture['HomeTeamID'].values[0], selected_fixture['AwayTeamID'].values[0]]
    # Get matches for each scores for teams against each other
    head2head = results_data[results_data['HomeTeamID'].isin(teams_ab) & 
                             results_data['AwayTeamID'].isin(teams_ab)]
    # create team 1 and team 2 list of goals
    team_a_scores = [head2head[head2head['HomeTeamID']==teams_ab[0]]['HomeScore'].values[0],
                     head2head[head2head['AwayTeamID']==teams_ab[0]]['AwayScore'].values[0]]
    team_b_scores = [head2head[head2head['AwayTeamID']==teams_ab[1]]['AwayScore'].values[0],
                     head2head[head2head['HomeTeamID']==teams_ab[1]]['HomeScore'].values[0]]
    # get all goals for teams
    team_a_goals = pd.concat([results_data[results_data['HomeTeamID']==teams_ab[0]]['HomeScore'],
                              results_data[results_data['AwayTeamID']==teams_ab[0]]['AwayScore']])
    team_b_goals = pd.concat([results_data[results_data['AwayTeamID']==teams_ab[1]]['AwayScore'],
                              results_data[results_data['HomeTeamID']==teams_ab[1]]['HomeScore']])
    
    return {'teams': teams_ab, 'head2head': [team_a_scores, team_b_scores], 'team_goals': [team_a_goals, team_b_goals]}

In [8]:
def get_alpha_beta(teams_data):
    """Take in Teams data and create alpha, beta
    for gamma dist team A, team B
    TODO: (1)check formula for calculating alpha, beta,
    (2) Check what data should go into alpha, beta for goals
    Is the distribution of goals scored before a good measure?
    """
    team_goals = teams_data['team_goals']
    alpha_a = team_goals[0].mean()**2 / team_goals[0].var()
    beta_a = team_goals[0].var() / team_goals[0].mean()

    alpha_b = team_goals[1].mean()**2 / team_goals[1].var()
    beta_b = team_goals[1].var() / team_goals[1].mean()

    return [[alpha_a, beta_a], [alpha_b, beta_b]]

In [9]:
## Single prediction for ID 2 Vs. 1
def fit_gp_model(teams_data, alpha_beta):
    """ Initialise model, fit priors w/ alpha, beta
    update with observed results
    """
    model = pm.Model()
    with model:
        mu_a = pm.Gamma('mu_a', alpha_beta[0][0], alpha_beta[0][1])
        mu_b = pm.Gamma('mu_b', alpha_beta[1][0], alpha_beta[1][1])
        goals_1 = pm.Poisson('goals_a', mu_a, observed=teams_data['head2head'][0])
        goals_2 = pm.Poisson('goals_b', mu_b, observed=teams_data['head2head'][1])
        trace = pm.sample(300)
    return model, trace

In [10]:
def predict_results(model, trace):
    """
    Sample posterior for predictions and return 1000 runs of
    goals for team A and team B
    TODO: Why do I get 2000 results from 1000 samples?
    """
    with model:
        post_pred = pm.sample_posterior_predictive(trace, samples=300)
    goals_a = post_pred['goals_a'].flatten()
    goals_b = post_pred['goals_b'].flatten()
    return [goals_a, goals_b]

In [11]:
def get_probabilities(teams_data, match_results):
    """Calculate the result probabilities for
    win, lose, draw for the fixture and format
    with the teams data
    TODO: sort out what is the right output format
    """
    win = np.mean(match_results[0] > match_results[1])
    lose = np.mean(match_results[0] < match_results[1])
    draw = np.mean(match_results[0] == match_results[1])
    result = {'HomeTeamID': [teams_data['teams'][0]],
                           'AwayTeamID': [teams_data['teams'][1]],
                           'HomeWin': [win],
                           'AwayWin': [lose],
                           'Draw': [draw]
                           }
    return result

In [12]:
# Example: get data
def run_pipeline(fixtures, results, index):
    """Run pipeline for fixture index"""
    selected_fix = fixtures[fixtures.index==index]
    season1 = results[results['SeasonID']==1]
    teams_data = get_goals_data(selected_fix, season1)
    alpha_beta = get_alpha_beta(teams_data)
    model, trace = fit_gp_model(teams_data, alpha_beta)
    match_results = predict_results(model, trace)
    results_df = get_probabilities(teams_data, match_results)
    print(f'Completed predictions for: {index}')
    return results_df, match_results

In [13]:
results['Points'] = results.apply(lambda row: results_points(row['HomeScore'], row['AwayScore']), axis=1)
results[['HomePoints', 'AwayPoints']] = pd.DataFrame(results['Points'].tolist(), index=results.index)
season1 = results[results['SeasonID']==1]
points_by_week_s1 = get_points_by_week(season1)
# Sum points over all weeks by Team
final_table_s1 = (points_by_week_s1.groupby(['ID']).agg({'Points': 'sum', 'For': 'sum', 'Against': 'sum',
                                                         'Gameweek': 'count'})
               .reset_index()
               .sort_values('Points', ascending=False)
               )
final_table_s1['GD'] = final_table_s1['For'] - final_table_s1['Against']
final_table_s1 = final_table_s1.reset_index().drop('index', axis=1)

final_table_s1 = pd.merge(final_table_s1, teams, left_on='ID', right_on='TeamID')
final_table_s1[['TeamName', 'Gameweek','Points', 'For', 'Against', 'GD']].head(5)

,TeamName,Gameweek,Points,For,Against,GD
0,Miami,54,138,159,41,118
1,Cincinnati,54,125,130,51,79
2,Baltimore,54,117,136,41,95
3,New York S,54,113,108,52,56
4,Boston,54,106,130,58,72


In [1]:
#list_dict = [run_pipeline(fixtures, results, i) for i in range(fixtures.shape[0])]

In [22]:
home = results[['Gameweek', 'HomeTeamID', 'HomeScore']]
home.columns = ['Gameweek', 'TeamID', 'Score']
away = results[['Gameweek', 'AwayTeamID', 'AwayScore']]
away.columns = ['Gameweek', 'TeamID', 'Score']
goals_df = pd.concat([home, away])

In [40]:
teamids = sorted(goals_df['TeamID'].drop_duplicates().to_list())
past_goals = {str(x): goals_df[goals_df['TeamID']==x]['Score'].to_list() for x in teamids}

In [42]:
model = pm.Model()

with model:
    alpha = pm.Exponential('alpha', lam=1)
    beta = pm.Exponential('beta', lam=1)
    
    mu = dict()
    goals = dict()
    for name, observed in past_goals.items():
        mu[name] = pm.Gamma('mu_'+ str(name), alpha, beta)
        goals[name] = pm.Poisson(name, mu[name], observed=observed)
        
    trace = pm.sample(500)#, nuts_kwargs=dict(target_accept=0.95))

C:\Users\chris\AppData\Local\Temp\ipykernel_12368\1071094365.py:13: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(500)#, nuts_kwargs=dict(target_accept=0.95))
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu_28, mu_27, mu_26, mu_25, mu_24, mu_23, mu_22, mu_21, mu_20, mu_19, mu_18, mu_17, mu_16, mu_15, mu_14, mu_13, mu_12, mu_11, mu_10, mu_9, mu_8, mu_7, mu_6, mu_5, mu_4, mu_3, mu_2, mu_1, beta, alpha]


Sampling 4 chains for 1_000 tune and 500 draw iterations (4_000 + 2_000 draws total) took 47 seconds.


In [43]:
with model:
    post_pred = pm.sample_posterior_predictive(trace, samples=10000)

In [48]:
goals_1 = post_pred['1'].flatten()
goals_2 = post_pred['2'].flatten()

In [49]:
win = np.mean(goals_1 > goals_2)
lose = np.mean(goals_1 < goals_2)
draw = np.mean(goals_1 == goals_2)

In [50]:
win, lose, draw

(0.5096379629629629, 0.2404388888888889, 0.24992314814814814)